<a href="https://colab.research.google.com/github/AdhamMahgoub2000/machine-learning-zoomcamp2025/blob/main/deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deep Learning Homework

## 1.Loading data set


In [1]:
import torch
import torchvision.models as models
from torchvision import transforms
import numpy as np
from PIL import Image

In [ ]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip data.zip

## 2. Reproducibility

In [3]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## 3. data set class


In [4]:
import os
from torch.utils.data import Dataset
from PIL import Image

class HairStyleDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

## 4. Developing the model

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class HairStyleMODEL(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=(3, 3),
            stride=1,
            padding=0
        )
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))

        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))   # Conv + ReLU
        x = self.pool(x)            # Max Pool
        x = torch.flatten(x, 1)     # Flatten
        x = F.relu(self.fc1(x))     # FC + ReLU
        x = self.fc2(x)  # Output + Sigmoid
        return x

In [6]:
model = HairStyleMODEL()

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.002,
    momentum=0.8
)
criterion = nn.BCEWithLogitsLoss()

In [15]:
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
         MaxPool2d-2           [-1, 32, 99, 99]               0
            Linear-3                   [-1, 64]      20,072,512
            Linear-4                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 11.96
Params size (MB): 76.57
Estimated Total Size (MB): 89.00
----------------------------------------------------------------


## 5. preprocessing the data


In [8]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [9]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


train_dataset = datasets.ImageFolder(root='data/train', transform=train_transforms)
validation_dataset   = datasets.ImageFolder(root='data/test', transform=train_transforms)

# Create DataLoaders with batch size
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
validation_loader   = DataLoader(validation_dataset, batch_size=32, shuffle=False, num_workers=2)

## 6. Training the model

In [10]:
num_epochs=10
device = torch.device("cuda" if torch.cuda.is_available() else
                      "mps" if getattr(torch.backends, 'mps', None) and torch.backends.mps.is_available() else
                      "cpu")
model.to(device)
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6549, Acc: 0.5962, Val Loss: 0.6146, Val Acc: 0.6368
Epoch 2/10, Loss: 0.5827, Acc: 0.6800, Val Loss: 0.5883, Val Acc: 0.6269
Epoch 3/10, Loss: 0.5261, Acc: 0.7400, Val Loss: 0.6199, Val Acc: 0.6368
Epoch 4/10, Loss: 0.4887, Acc: 0.7600, Val Loss: 0.5953, Val Acc: 0.6567
Epoch 5/10, Loss: 0.4474, Acc: 0.7900, Val Loss: 0.6799, Val Acc: 0.6418
Epoch 6/10, Loss: 0.4289, Acc: 0.8113, Val Loss: 0.6584, Val Acc: 0.6965
Epoch 7/10, Loss: 0.3365, Acc: 0.8625, Val Loss: 0.6772, Val Acc: 0.6766
Epoch 8/10, Loss: 0.3265, Acc: 0.8562, Val Loss: 0.6956, Val Acc: 0.6866
Epoch 9/10, Loss: 0.2501, Acc: 0.8962, Val Loss: 0.6735, Val Acc: 0.7363
Epoch 10/10, Loss: 0.1763, Acc: 0.9413, Val Loss: 0.7003, Val Acc: 0.7363


## 7. data augmentaion

In [13]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),                           # Random rotation ±50 degrees
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),  # Random crop + resize
    transforms.RandomHorizontalFlip(),                       # Random horizontal flip
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [14]:
num_epochs=10
device = torch.device("cuda" if torch.cuda.is_available() else
                      "mps" if getattr(torch.backends, 'mps', None) and torch.backends.mps.is_available() else
                      "cpu")
model.to(device)
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6452, Acc: 0.6425, Val Loss: 0.5997, Val Acc: 0.6866
Epoch 2/10, Loss: 0.5949, Acc: 0.6438, Val Loss: 0.5753, Val Acc: 0.6617
Epoch 3/10, Loss: 0.5923, Acc: 0.6625, Val Loss: 0.6223, Val Acc: 0.6766
Epoch 4/10, Loss: 0.5665, Acc: 0.6825, Val Loss: 0.6100, Val Acc: 0.7114
Epoch 5/10, Loss: 0.5659, Acc: 0.7075, Val Loss: 0.6316, Val Acc: 0.6965
Epoch 6/10, Loss: 0.5529, Acc: 0.7013, Val Loss: 0.5779, Val Acc: 0.6766
Epoch 7/10, Loss: 0.5430, Acc: 0.7037, Val Loss: 0.5744, Val Acc: 0.7065
Epoch 8/10, Loss: 0.5370, Acc: 0.7025, Val Loss: 0.6103, Val Acc: 0.6617
Epoch 9/10, Loss: 0.5446, Acc: 0.7200, Val Loss: 0.5920, Val Acc: 0.6567
Epoch 10/10, Loss: 0.5369, Acc: 0.7275, Val Loss: 0.5537, Val Acc: 0.7065
